<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
    
Cette étude a été réalisée dans le cadre du 6ème projet de ma formation Datascientist dispensée en MOOC par 

<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>.




<p></p><p></p><p></p>





**Le problème posé :**


*Stack Overflow est un site célèbre de question-réponses liées au développement informatique. Pour poser une question sur ce site, il faut entrer plusieurs tags de manière à retrouver facilement la question par la suite. Pour les utilisateurs expérimentés cela ne pose pas de problème, mais pour les nouveaux utilisateurs, il serait judicieux de suggérer quelques tags relatifs à la question posée.*

*Amateur de Stack Overflow, qui vous a souvent sauvé la mise, vous décidez d'aider la communauté en retour. Pour cela, vous développez un système de suggestion de tags pour le site. Celui-ci prendra la forme d’un algorithme de machine learning qui assigne automatiquement plusieurs tags pertinents à une question.*



**Solutions mises en oeuvre**

Les solutions de suggestion de tags présentés ici se basent sur des modèles supervisés de machine learning.

Les modèmes mis en oeuvre : 
    * Naive Multinomial Baysien

Ce notebook utilise les données issues des notebooks : 


**P6_DadaAnalysis.ipynb**

**P6_UnsupervizedMethods.ipynb**



In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# 4. Supervized methods

**Loading TF-IDF operator and CSR matrix**

In [3]:
import p5_util
file_name="./data/csr_matrix_tdif_ngram_2_2.dump"
csr_matrix = p5_util.object_load(file_name)
print(csr_matrix.shape)

file_name="./data/vectorizer_tdif_ngram_2_2.dump"
vectorizer = p5_util.object_load(file_name)

p5_util.object_load : fileName= ./data/csr_matrix_tdif_ngram_2_2.dump
(16359, 898)
p5_util.object_load : fileName= ./data/vectorizer_tdif_ngram_2_2.dump


## 4.1. Test dataset vectorization

In [15]:
import pandas as pd
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
df_sof_test=pd.read_csv(file_name)
print(df_sof_test.shape)

(16359, 5)


**Test dataset standardization : applies on `Body` column**

In [16]:
df_sof_test.columns

Index(['Id', 'PostTypeId', 'Body', 'Title', 'Tags'], dtype='object')

In [17]:
import p6_util
ser_sof_body_test = p6_util.p6_df_standardization(df_sof_test.Body)


Cleaning text in-between markers <code></code> markers...

Cleaning LXML markers...

Remove verbs from sentences...

Filtering alpha-numeric words from sentences...

Removing stopwords...

Lemmatization ...


In [18]:
csr_matrix_tdif_ngram_2_2_test = vectorizer.transform(ser_sof_body_test)

In [20]:
csr_matrix_tdif_ngram_2_2_test.A.shape, df_sof_test.shape

((16359, 898), (16359, 5))

**Save of test CSR matrix**

In [21]:
file_name="./data/csr_matrix_tdif_ngram_2_2_test.dump"
p5_util.object_dump(csr_matrix_tdif_ngram_2_2_test,file_name)

## 4.2. Target processing : list of TAGS are encoded

### 4.2.1. Target : TAGs for train dataset are loaded and formated

In [1]:
import pandas as pd
file_name="./data/train_StackOverFlow_BodyTitleTags.csv"
ser_sof_train_tags=pd.read_csv(file_name).Tags.copy()
ser_sof_train_tags.shape


(24604,)

**Series is resized to fit with Test dataset**

In [4]:
ser_sof_train_tags = ser_sof_train_tags[:csr_matrix.shape[0]]
ser_sof_train_tags.shape

(16359,)

**Tags are converted into a lists of words.**

In [5]:
import numpy as np
from scipy import sparse
import p6_util
def p6_encode_ser_tag_2_csrmatrix(ser_tag):
    #-----------------------------------------------------------------------------------------
    # Markers '<' and '>' are removed from tags.
    #-----------------------------------------------------------------------------------------
    ser_tag = ser_tag.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>')

    #-----------------------------------------------------------------------------------------
    # A unique list of all TAGs is built : this is the vocabulary for TAGs**
    # This list is supposed to be completed enough for covering test tags dataset.
    #-----------------------------------------------------------------------------------------
    list_all_tags = p6_util.p6_get_list_all_tag(ser_tag)
    
    #-----------------------------------------------------------------------------------------
    # For each row, TAGs represented as a list of elementaries TAG are encoded
    # Each row that is a string of TAGs is splitted into a list of elementary TAGs
    # All rows are aggregated into a list
    #-----------------------------------------------------------------------------------------
    list_list_encoded_row = p6_util.p6_encode_target(list_all_tags, ser_tag.tolist())

    #-----------------------------------------------------------------------------------------
    # Conversion into CSR matrix fro easyness of computation
    #-----------------------------------------------------------------------------------------
    csr_matrix_encoded_tag=sparse.csr_matrix(np.array(list_list_encoded_row))
    return csr_matrix_encoded_tag

In [6]:
csr_matrix_encoded_train_tags = p6_encode_ser_tag_2_csrmatrix(ser_sof_train_tags)

**Save of encoded train-dataset target**

In [7]:
import p5_util

file_name="./data/csr_matrix_encoded_train_tags.dump"
p5_util.object_dump(csr_matrix_encoded_train_tags,file_name)
print(csr_matrix_encoded_train_tags.shape)

(16359, 5155)


## 4.3. Applying Naive Bayes classifier

### 4.3.1. Training classifier

**Loading TF-IDF data-set containing trained vectorized questions**

In [22]:
import p5_util

file_name="./data/csr_matrix_tdif_ngram_2_2.dump"
csr_matrix_tdif_ngram_2_2 = p5_util.object_load(file_name)
print(csr_matrix_tdif_ngram_2_2.shape)

p5_util.object_load : fileName= ./data/csr_matrix_tdif_ngram_2_2.dump
(16359, 898)


**Loading target train data-set containing encoded TAGs**

In [24]:
import p5_util

file_name="./data/csr_matrix_encoded_train_tags.dump"
csr_matrix_encoded_train_target = p5_util.object_load(file_name)
print(csr_matrix_encoded_train_target.shape)

p5_util.object_load : fileName= ./data/csr_matrix_encoded_train_tags.dump
(16359, 5155)


**Training classifier**

one versus Rest leads having one classifier per class.

`OneVsRestClassifier` classifier is used because of multiple classes. 

Then, one classifier per class is fitted.



In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

model = OneVsRestClassifier(MultinomialNB(), n_jobs=4).fit(csr_matrix_tdif_ngram_2_2, csr_matrix_encoded_train_target)


### 4.3.2.Classifier evaluation

**Loading test data-set**

In [26]:
file_name="./data/csr_matrix_tdif_ngram_2_2_test.dump"
csr_matrix_tdif_ngram_2_2_test = p5_util.object_load(file_name)
csr_matrix_tdif_ngram_2_2_test.shape

p5_util.object_load : fileName= ./data/csr_matrix_tdif_ngram_2_2_test.dump


(16359, 898)

In [27]:
model

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
          n_jobs=4)

**Predictions and probabilities**

In [28]:
csr_matrix_encoded_predict_tags = model.predict(csr_matrix_tdif_ngram_2_2_test)

In [51]:
import numpy as np
np.where(csr_matrix_encoded_predict_tags[1000].A!=0)

(array([], dtype=int64), array([], dtype=int64))

**<font color=blue>P({Tag_i}|Question)</font>**

In [53]:
csr_matrix_encoded_predict_proba_tags = model.predict_proba(csr_matrix_tdif_ngram_2_2_test)

In [56]:
import numpy as np
np.where(csr_matrix_encoded_predict_proba_tags[1000]!=0)
csr_matrix_encoded_predict_proba_tags[1000]

array([6.11284308e-05, 4.76801760e-03, 1.83385292e-04, ...,
       6.11284308e-05, 6.11284308e-05, 5.50155877e-04])

**Compute accuracy**

In [35]:
df_sof_test.columns
ser_sof_test_tags=df_sof_test['Tags']
ser_sof_test_tags.shape

(16359,)

Tests TAGs are encoed then converted into CSR matrix

In [36]:
csr_matrix_encoded_test_tags = p6_encode_ser_tag_2_csrmatrix(ser_sof_test_tags)

In [44]:
csr_matrix_tdif_ngram_2_2_test.shape, csr_matrix_encoded_test_tags.shape,csr_matrix_encoded_predict_tags.shape

((16359, 898), (16359, 5547), (16359, 5155))

In [46]:
score = model.score(csr_matrix_tdif_ngram_2_2_test ,csr_matrix_encoded_test_tags[:,:5155])
print("Mean score for Multinomial Naive Bayse classifier : "+str(score))

Mean score for Multinomial Naive Bayse classifier : 0.010086191087474784
